In [1]:
%load_ext autoreload
%autoreload 2

In [25]:
import torch
import numpy as np
import json
from tqdm import tqdm
import random

import os
import sys
sys.path.append('..')

from relations import estimate
from util import model_utils
from dsets.counterfact import CounterFactDataset
from util import nethook

In [3]:
MODEL_NAME = "gpt2-xl"  # gpt2-{medium,large,xl} or EleutherAI/gpt-j-6B
mt = model_utils.ModelAndTokenizer(MODEL_NAME, low_cpu_mem_usage=False)

model = mt.model
tokenizer = mt.tokenizer
tokenizer.pad_token = tokenizer.eos_token

In [6]:
test_cases = [
    ("The Great Wall", -1, "China"),
    ("Niagara Falls", -2, "Canada"),
    ("Valdemarsvik", -1, "Sweden"),
    ("Kyoto University", -2, "Japan"),
    ("Hattfjelldal", -1, "Norway"),
    ("Ginza", -1, "Japan"),
    ("Sydney Hospital", -2, "Australia"),
    ("Mahalangur Himal", -1, "Nepal"),
    ("Higashikagawa", -1, "Japan"),
    ("Trento", -1, "Italy"),
    ("Taj Mahal", -1, "India")
]

def check_with_test_cases(relation_operator):
    for subject, subject_token_index, target in test_cases:
        objects = relation_operator(
            subject,
            subject_token_index=subject_token_index,
            device=model.device,
            return_top_k=5,
        )
        print(f"{subject}, target: {target}   ==>   predicted: {objects}")

In [4]:
###########################################################################
relation_dct = {
    'P17': {'relation': '{} is located in the country of', 'correct_predict': "P17/correct_prediction_P17__gpt2-xl.json", 'cached_JB': "P17/cached_jacobians/jacobian_calculations__all_sub_toks__P17__layer_25.npz"},
    'P641': {'relation': '{} plays the sport of', 'correct_predict': "P641/correct_prediction_P641__gpt2-xl.json", 'cached_JB': "P641/jacobian_calculations__all_sub_toks__P641__layer_25.npz"},
    'P103': {'relation': 'The mother tongue of {} is', 'correct_predict': "P103/correct_prediction_P103__gpt2-xl.json", 'cached_JB': "P103/jacobian_calculations__all_sub_toks__P103__layer_25.npz"},
    'P176': {'relation': '{} is produced by', 'correct_predict': "P176/correct_prediction_P176__gpt2-xl.json", 'cached_JB': "P176/jacobian_calculations__all_sub_toks__P176__layer_25.npz"},
}
###########################################################################

In [5]:
good_cases = {
    'P17':[
        ('Etobicoke North', 2, 'Canada'),
        ('Menangle Park', 1, 'Australia'),
        ('Miramichi Centre', 2, 'Canada'),
        ('Campeche Bank', 2, 'Mexico'),
        ('Naumburg (Saale)', 5, 'Germany'),
        ('plaza de Cibeles', 5, 'Spain'),
    ]
}

In [7]:
#####################################################
relation_id = "P17"
#####################################################

In [11]:
relation_collection = []
for subject, idx, target in good_cases[relation_id]:
    relation_operator = estimate.estimate_relation_operator(
        model, tokenizer,
        subject, relation_dct[relation_id]['relation'],
        layer=25, subject_token_index = idx,
        device=model.device,
    )
    print(relation_operator.misc)   
    # check_with_test_cases(relation_operator)
    # print()
    relation_collection.append(relation_operator)

{'Jh_norm': 23.247802734375, 'bias_norm': 486.58489990234375, 'h_info': {'h_index': 2, 'token_id': 20803, 'token': 'obic'}, 'consider_residual': False}
{'Jh_norm': 22.141036987304688, 'bias_norm': 522.3410034179688, 'h_info': {'h_index': 1, 'token_id': 9248, 'token': 'angle'}, 'consider_residual': False}
{'Jh_norm': 26.415491104125977, 'bias_norm': 430.8843994140625, 'h_info': {'h_index': 2, 'token_id': 16590, 'token': 'ichi'}, 'consider_residual': False}
{'Jh_norm': 46.61355209350586, 'bias_norm': 443.0304870605469, 'h_info': {'h_index': 2, 'token_id': 2395, 'token': 'che'}, 'consider_residual': False}
{'Jh_norm': 31.205049514770508, 'bias_norm': 517.58056640625, 'h_info': {'h_index': 5, 'token_id': 1000, 'token': 'ale'}, 'consider_residual': False}
{'Jh_norm': 17.898963928222656, 'bias_norm': 446.3467102050781, 'h_info': {'h_index': 5, 'token_id': 274, 'token': 'es'}, 'consider_residual': False}


In [40]:
for r1 in relation_collection:
    arr = []
    for r2 in relation_collection:
        # print(r1.bias.norm().item(), r2.bias.norm().item(), torch.dist(r1.bias, r2.bias).item())
        arr.append(np.round(torch.dist(r1.weight, r2.weight).item(), 3))
    print(r1.weight.norm().item(), arr)

12.594758987426758 [0.0, 16.952, 18.875, 27.862, 17.031, 19.042]
17.02851104736328 [16.952, 0.0, 19.567, 28.279, 18.751, 20.4]
18.64788246154785 [18.875, 19.567, 0.0, 27.658, 20.576, 20.895]
28.792652130126953 [27.862, 28.279, 27.658, 0.0, 28.578, 24.671]
15.542061805725098 [17.031, 18.751, 20.576, 28.578, 0.0, 19.969]
18.99873924255371 [19.042, 20.4, 20.895, 24.671, 19.969, 0.0]


In [20]:
precalculated_relations = np.load(
    "P17/cached_jacobians/jacobian_calculations__all_sub_toks__P17__layer_25.npz",
    allow_pickle= True
)['jacobians']

In [21]:
len(precalculated_relations)

266

In [22]:
bad_cases = []
for c in precalculated_relations:
    bad_cases += c['all_weights_and_biases']

In [39]:
relation_collection_bad = random.sample(bad_cases, 6)

for r1 in relation_collection_bad:
    r1_bias = torch.tensor(r1['weight'])
    arr = []
    for r2 in relation_collection_bad:
        r2_bias = torch.tensor(r2['weight'])
        # print(r1_bias.norm().item(), r2_bias.norm().item(), torch.dist(r1_bias, r2_bias).item())
        arr.append(np.round(torch.dist(r1_bias, r2_bias).item(), 3))
    print(r1_bias.norm().item(), arr)

4.320530891418457 [0.0, 6.46, 10.998, 9.945, 6.528, 13.408]
5.322466850280762 [6.46, 0.0, 11.687, 10.626, 7.393, 14.152]
11.044798851013184 [10.998, 11.687, 0.0, 12.696, 11.647, 15.394]
9.669387817382812 [9.945, 10.626, 12.696, 0.0, 10.527, 14.935]
5.642831325531006 [6.528, 7.393, 11.647, 10.527, 0.0, 13.834]
13.551694869995117 [13.408, 14.152, 15.394, 14.935, 13.834, 0.0]
